In [59]:
!pip install ipywidgets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 2.9 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 3.4 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 4.4 MB/s eta 0:00:005.2 MB/s eta 0:00:01


In [62]:
!jupyter nbextension enable --py widgetsnbextension


Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [63]:
%%time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier, Pool, cv

CPU times: user 8.31 ms, sys: 6.1 ms, total: 14.4 ms
Wall time: 26.2 ms


In [6]:
%%time
data_set = pd.read_csv('UNR-IDD.csv')


CPU times: user 81.1 ms, sys: 3.88 ms, total: 85 ms
Wall time: 84.3 ms


In [7]:
data_set = data_set.sample(frac=1).reset_index(drop=True)

In [8]:
data_set

,Switch ID,Port Number,Received Packets,Received Bytes,Sent Bytes,Sent Packets,Port alive Duration (S),Packets Rx Dropped,Packets Tx Dropped,Packets Rx Errors,...,Unknown Load/Latest,Latest bytes counter,is_valid,Table ID,Active Flow Entries,Packets Looked Up,Packets Matched,Max Size,Label,Binary Label
0,of:0000000000000005,Port#:1,722,45627,6356730,1006,76,0,0,0,...,33912,486,True,0,6,1990,1884,-1,TCP-SYN,Attack
1,of:0000000000000005,Port#:1,372,6322748,6368651,1220,81,0,0,0,...,6348983,513,True,0,7,1759,1652,-1,TCP-SYN,Attack
2,of:0000000000000008,Port#:1,312,33271,25374131,119861,221,0,0,0,...,0,0,True,0,7,121649,121534,-1,PortScan,Attack
3,of:0000000000000006,Port#:3,6750,120130653,259582,1906,2826,0,0,0,...,0,0,True,0,6,17311,17177,-1,Blackhole,Attack
4,of:0000000000000001,Port#:3,1584,19028943,100037,780,1042,0,0,0,...,0,0,True,0,4,4502,4332,-1,Diversion,Attack
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37406,of:0000000000000006,Port#:2,96,12693,76477,1278,96,0,0,0,...,63234,529,True,0,7,4555,4456,-1,TCP-SYN,Attack
37407,of:0000000000000009,Port#:1,2842,166383,19344,149,161,0,0,0,...,0,0,True,0,6,3797,3681,-1,TCP-SYN,Attack
37408,of:000000000000000b,Port#:2,361,33500,12625319,489,146,0,0,0,...,6303303,262641,True,0,6,850,786,-1,TCP-SYN,Attack
37409,of:000000000000000b,Port#:2,315,6322345,27152,262,156,0,0,0,...,0,0,True,0,4,580,517,-1,PortScan,Attack


In [9]:
categorical_columns = data_set.select_dtypes(include=['object']).columns

In [13]:
numerical_columns = data_set.select_dtypes(include=['number']).columns

In [10]:
for col in categorical_columns:
    unique_values = data_set[col].unique()
    print(f"Valeurs uniques de la colonne '{col}':")
    print(unique_values)
    print("-" * 40)

Valeurs uniques de la colonne 'Switch ID':
['of:0000000000000005' 'of:0000000000000008' 'of:0000000000000006'
 'of:0000000000000001' 'of:0000000000000003' 'of:000000000000000c'
 'of:000000000000000b' 'of:0000000000000004' 'of:0000000000000009'
 'of:0000000000000007' 'of:000000000000000a' 'of:0000000000000002']
----------------------------------------
Valeurs uniques de la colonne 'Port Number':
['Port#:1' 'Port#:3' 'Port#:4' 'Port#:2']
----------------------------------------
Valeurs uniques de la colonne 'Label':
['TCP-SYN' 'PortScan' 'Blackhole' 'Diversion' 'Normal' 'Overflow']
----------------------------------------
Valeurs uniques de la colonne 'Binary Label':
['Attack' 'Normal']
----------------------------------------


In [14]:
for col in numerical_columns:
    unique_values = data_set[col].unique()
    print(f"Valeurs uniques de la colonne '{col}':")
    print(unique_values)
    print("-" * 40)

Valeurs uniques de la colonne 'Received Packets':
[   722    372    312 ...  66529   7770 136836]
----------------------------------------
Valeurs uniques de la colonne 'Received Bytes':
[   45627  6322748    33271 ...    33500  6322345 12682754]
----------------------------------------
Valeurs uniques de la colonne 'Sent Bytes':
[ 6356730  6368651 25374131 ... 12625319    27152  6379700]
----------------------------------------
Valeurs uniques de la colonne 'Sent Packets':
[  1006   1220 119861 ...   6458 238606   3452]
----------------------------------------
Valeurs uniques de la colonne 'Port alive Duration (S)':
[ 76  81 221 ... 362 322 215]
----------------------------------------
Valeurs uniques de la colonne 'Packets Rx Dropped':
[0]
----------------------------------------
Valeurs uniques de la colonne 'Packets Tx Dropped':
[0]
----------------------------------------
Valeurs uniques de la colonne 'Packets Rx Errors':
[0]
----------------------------------------
Valeurs unique

In [20]:
def trouver_variables_non_variables(dataset):
    """
    Identifie les variables catégorielles et numériques non-variées dans un DataFrame.

    Args:
    - dataset (DataFrame): Le DataFrame contenant les données.

    Returns:
    - variables_non_variables (list): Une liste des colonnes qui n'ont qu'une seule valeur unique.
    """
    variables_non_variables = []
    for col in dataset.columns:
        # Vérifier si la colonne a une seule valeur unique
        if dataset[col].nunique() == 1:
            variables_non_variables.append(col)
    return variables_non_variables


In [21]:
variables_non_variables = trouver_variables_non_variables(data_set)
variables_non_variables

['Packets Rx Dropped',
 'Packets Tx Dropped',
 'Packets Rx Errors',
 'Packets Tx Errors',
 'Delta Packets Rx Dropped',
 ' Delta Packets Tx Dropped',
 'Delta Packets Rx Errors',
 'Delta Packets Tx Errors',
 'is_valid',
 'Table ID',
 'Max Size']

In [30]:
data_set =data_set.drop(variables_non_variables, axis = 1)

In [31]:
columns_to_drop = ['Switch ID', 'Port Number']

In [33]:
#data_set.drop(columns=columns_to_drop, inplace=True)

In [34]:
X = data_set.drop(["Label"], axis = 1)
y = data_set["Label"]

In [35]:
X

,Received Packets,Received Bytes,Sent Bytes,Sent Packets,Port alive Duration (S),Delta Received Packets,Delta Received Bytes,Delta Sent Bytes,Delta Sent Packets,Delta Port alive Duration (S),Connection Point,Total Load/Rate,Total Load/Latest,Unknown Load/Rate,Unknown Load/Latest,Latest bytes counter,Active Flow Entries,Packets Looked Up,Packets Matched,Binary Label
0,722,45627,6356730,1006,76,97,5578,5578,97,4,1,486,33912,486,33912,486,6,1990,1884,Attack
1,372,6322748,6368651,1220,81,4,556,5686,99,5,1,513,6348983,513,6348983,513,7,1759,1652,Attack
2,312,33271,25374131,119861,221,4,556,556,4,5,1,0,0,0,0,0,7,121649,121534,Attack
3,6750,120130653,259582,1906,2826,97,3676837,556,4,5,4,0,0,0,0,0,6,17311,17177,Attack
4,1584,19028943,100037,780,1042,3,320,320,3,5,4,0,0,0,0,0,4,4502,4332,Attack
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37406,96,12693,76477,1278,96,2,278,5516,99,5,2,529,63234,529,63234,529,7,4555,4456,Attack
37407,2842,166383,19344,149,161,97,5408,278,2,5,1,0,0,0,0,0,6,3797,3681,Attack
37408,361,33500,12625319,489,146,46,3609,2626975,75,5,2,262641,6303303,262641,6303303,262641,6,850,786,Attack
37409,315,6322345,27152,262,156,4,560,560,4,4,2,0,0,0,0,0,4,580,517,Attack


In [37]:
categorical_columns = X.select_dtypes(include=['object']).columns
categorical_columns

Index(['Binary Label'], dtype='object')

In [91]:
categorical_features_indices = [X.columns.get_loc(col) for col in X.select_dtypes(include=['object', 'category']).columns]

In [92]:
%%time
# Liste de toutes les colonnes de X
Vars = list(X.columns)

# Identification de la première colonne ayant des valeurs manquantes
Nan_Vars = Vars[X.isnull().sum().values.nonzero()[0][0]]

# Affichage de la première colonne avec des valeurs manquantes
Nan_Vars


IndexError: index 0 is out of bounds for axis 0 with size 0

In [93]:
train_pool = Pool(data=X, label=y, cat_features=categorical_features_indices)
test_pool = Pool(data=X, label=y, cat_features=categorical_features_indices)

In [96]:
model_params = {
    'iterations': 2000,
    'learning_rate': 0.1,
    'depth': 8,
    'loss_function': 'MultiClass',
    'eval_metric': 'Accuracy',
    'verbose': 100,
    'random_seed': 42
}

In [97]:
model = CatBoostClassifier(**model_params)

In [98]:
%%time
#Ajuster le modèle et évaluer la performance.
model.fit(train_pool, eval_set=test_pool,plot=True)
y_pred_probaCB = model.predict_proba(X)

# Calculer l'AUC ROC pour la classification multiclasse
auc = roc_auc_score(y, y_pred_probaCB, multi_class='ovr')  # Utilisez 'ovr' pour la stratégie multiclasse
print("AUC: ", auc)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.7303734	test: 0.7303734	best: 0.7303734 (0)	total: 57.4ms	remaining: 1m 54s
100:	learn: 0.8991473	test: 0.8991473	best: 0.8991473 (100)	total: 5.21s	remaining: 1m 38s
200:	learn: 0.9311967	test: 0.9311967	best: 0.9311967 (200)	total: 10.3s	remaining: 1m 31s
300:	learn: 0.9495603	test: 0.9495603	best: 0.9496672 (299)	total: 15.8s	remaining: 1m 29s
400:	learn: 0.9611612	test: 0.9611612	best: 0.9611612 (400)	total: 21s	remaining: 1m 23s
500:	learn: 0.9697148	test: 0.9697148	best: 0.9697415 (499)	total: 26s	remaining: 1m 17s
600:	learn: 0.9749272	test: 0.9749272	best: 0.9749539 (599)	total: 31.5s	remaining: 1m 13s
700:	learn: 0.9800861	test: 0.9800861	best: 0.9800861 (699)	total: 36.8s	remaining: 1m 8s
800:	learn: 0.9836144	test: 0.9836144	best: 0.9836412 (794)	total: 41.9s	remaining: 1m 2s
900:	learn: 0.9866617	test: 0.9866617	best: 0.9866884 (898)	total: 47.6s	remaining: 58.1s
1000:	learn: 0.9886130	test: 0.9886130	best: 0.9886130 (1000)	total: 53s	remaining: 52.9s
1100:	lear

In [99]:
%%time
cv_results = cv(
    pool=train_pool,
    params=model_params,
    fold_count=8,
    shuffle=True,
    partition_random_seed=42,
    plot=True  # Affichage du graphique de validation croisée
)

# Afficher les résultats de la validation croisée
print("CV Results:\n", cv_results)

# Exemple de calcul de l'AUC ROC pour chaque fold
auc_scores = cv_results['test-Accuracy-mean']
mean_auc = np.mean(auc_scores)
print("Mean Accuracy:", mean_auc)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Training on fold [0/8]
0:	learn: 0.7342356	test: 0.7290019	best: 0.7290019 (0)	total: 61.6ms	remaining: 2m 3s
100:	learn: 0.8964011	test: 0.8813849	best: 0.8822398 (98)	total: 5.25s	remaining: 1m 38s
200:	learn: 0.9301295	test: 0.9089549	best: 0.9089549 (200)	total: 10.7s	remaining: 1m 35s
300:	learn: 0.9505683	test: 0.9207096	best: 0.9213507 (298)	total: 16.6s	remaining: 1m 33s
400:	learn: 0.9619027	test: 0.9266937	best: 0.9266937 (399)	total: 21.7s	remaining: 1m 26s
500:	learn: 0.9698766	test: 0.9316093	best: 0.9318230 (499)	total: 26.5s	remaining: 1m 19s
600:	learn: 0.9762618	test: 0.9348151	best: 0.9348151 (551)	total: 31.6s	remaining: 1m 13s
700:	learn: 0.9811194	test: 0.9373798	best: 0.9373798 (698)	total: 36.9s	remaining: 1m 8s
800:	learn: 0.9850299	test: 0.9388758	best: 0.9403719 (761)	total: 41.7s	remaining: 1m 2s
900:	learn: 0.9875351	test: 0.9403719	best: 0.9407993 (896)	total: 46.8s	remaining: 57.1s
1000:	learn: 0.9897654	test: 0.9414405	best: 0.9416542 (943)	total: 52.3s	r

300:	learn: 0.9484344	test: 0.9161677	best: 0.9170231 (293)	total: 20.6s	remaining: 1m 56s
400:	learn: 0.9620590	test: 0.9223695	best: 0.9230111 (394)	total: 28.1s	remaining: 1m 51s
500:	learn: 0.9702765	test: 0.9281437	best: 0.9283576 (499)	total: 34.5s	remaining: 1m 43s
600:	learn: 0.9755919	test: 0.9298546	best: 0.9304962 (536)	total: 41.6s	remaining: 1m 36s
700:	learn: 0.9797464	test: 0.9337040	best: 0.9337040 (694)	total: 48.2s	remaining: 1m 29s
800:	learn: 0.9832595	test: 0.9341317	best: 0.9349872 (779)	total: 55.3s	remaining: 1m 22s
900:	learn: 0.9858256	test: 0.9366980	best: 0.9371257 (889)	total: 1m 2s	remaining: 1m 16s
1000:	learn: 0.9878112	test: 0.9390505	best: 0.9390505 (992)	total: 1m 10s	remaining: 1m 10s
1100:	learn: 0.9899190	test: 0.9401198	best: 0.9405475 (1086)	total: 1m 18s	remaining: 1m 3s
1200:	learn: 0.9917214	test: 0.9405475	best: 0.9418306 (1140)	total: 1m 25s	remaining: 57.1s
1300:	learn: 0.9930655	test: 0.9409752	best: 0.9418306 (1140)	total: 1m 33s	remainin

In [84]:
print(X)

       Received Packets  Received Bytes  Sent Bytes  Sent Packets  \
0                   722           45627     6356730          1006   
1                   372         6322748     6368651          1220   
2                   312           33271    25374131        119861   
3                  6750       120130653      259582          1906   
4                  1584        19028943      100037           780   
...                 ...             ...         ...           ...   
37406                96           12693       76477          1278   
37407              2842          166383       19344           149   
37408               361           33500    12625319           489   
37409               315         6322345       27152           262   
37410              1000        12682754     6379700           828   

       Port alive Duration (S)  Delta Received Packets  Delta Received Bytes  \
0                           76                      97                  5578   
1          